In [0]:
from google.colab import drive
import json  
import pandas as pd  
import os
import glob
import re
import datetime
import string
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
nltk.download('punkt')
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Text Analytics Project')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data_US = pd.read_csv('USvideos.csv')

In [0]:
data_US.columns.values

array(['video_id', 'trending_date', 'title', 'channel_title',
       'category_id', 'publish_time', 'tags', 'views', 'likes',
       'dislikes', 'comment_count', 'thumbnail_link', 'comments_disabled',
       'ratings_disabled', 'video_error_or_removed', 'description'],
      dtype=object)

In [0]:
#views, likes, dislikes, comment_count
data_US['ratio'] = data_US['likes']/(data_US['likes']+data_US['dislikes'])

In [0]:
data_US.loc[data_US['ratio']>= 0.98, 'Rating_Category'] = 2
data_US.loc[(data_US['ratio']< 0.98) & (data_US['ratio']>=0.94), 'Rating_Category'] = 1
data_US.loc[data_US['ratio']< 0.94, 'Rating_Category'] = 0

In [0]:
use_data = data_US[['title', 'channel_title', 'views', 'likes', 'dislikes', 'comment_count', 'ratio', 'Rating_Category']]
use_data = use_data.dropna()

In [0]:
len(use_data[use_data['Rating_Category']==0])

11797

In [0]:
## Function to normalize text
def text_norm(tokenized_sentence):
    """ Takes in a tokenized text to normalize by removing puncuation, numbers, etc. and returns normalized tokens"""
    corpus_normalized=[]
    for document in tokenized_sentence: 
        normalized_sentences=[]
        for i in document:
            #get rid of punct, white space, numbers, etc.
            sent = i.lower() #convert to lowercase
            sent = re.sub(r'\d+', '', sent) 
            sent = sent.translate(str.maketrans('', '', string.punctuation))
            sent = sent.strip() 
            sent = re.sub(r'(?:\n|\s+|\t)', ' ', sent) 
            normalized_sentences.append(sent) 
        normalized_sentences = [i for i in normalized_sentences if i] # remove empty string tokens
        corpus_normalized.append(normalized_sentences) #append sublists all to one list
    return corpus_normalized  #list of lists

def nltk_word_token(d):
    """ Takes in corpus list and returns a list of words within each document (aka list of list). """
    tokenized_word = []
    for document in d:
        word = word_tokenize(document)
        tokenized_word.append(word)
    return tokenized_word

def nltk_stem(tokenized_word):
    """Takes in corpus list and appends stemmed words to a list that is returned. """
    ps = PorterStemmer()
    stem_lists=[]
    for d in tokenized_word:
        stemmed_words=[]
        for word in d: 
            stemmed_words.append(ps.stem(word))
        stem_lists.append(stemmed_words)
    return stem_lists

In [0]:
title_list = use_data['title'].tolist()
rating_cat_list = use_data['Rating_Category'].tolist()



# now map our tokenize function to each element of the list of texts
tokenized_texts=nltk_word_token(title_list)
normalized_tokens=text_norm(tokenized_texts)
stemmed_tokens = nltk_stem(normalized_tokens)

titles = []
for title in stemmed_tokens:
    titles.append(' '.join(title))
df1 = pd.DataFrame({'Rating_Category': rating_cat_list, 'Title': titles})
df1['Rating_Category'] = df1['Rating_Category'].astype(int)


In [0]:
def try_SVM(df, ngram_range, loss_list, C_list):
  with open('Proj_SVM_model.txt', "w") as f:

    for i in ngram_range:
      for j in loss_list:
        for w in C_list: 
            tfidf = TfidfVectorizer(sublinear_tf=True, min_df=100, norm='l2', ngram_range=i, stop_words='english')
            features = tfidf.fit_transform(df.Title.tolist())
            saved = tfidf.fit(df.Title.tolist())
            labels = df.Rating_Category
            tfidf_name = f"proj_tfidf_{i}_{j}_{w}.pkl"
            pickle.dump(saved, open(tfidf_name, 'wb'))


            model = LinearSVC(loss=j, C=w, random_state=132)
            X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.30, random_state=132)
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            class_report = metrics.classification_report(y_test, y_pred)

            model_name = f"proj_model_{i}_{j}_{w}.pkl"
            pickle.dump(model, open(model_name, 'wb'))


            print("\nFor {0} grams and {1} penalty and {2} regularization strength, the size of the feature matrix is:".format(i, j, w), file=f)
            print(features.shape, file=f)
            print(class_report, file=f)
            print("\n", file=f)


In [0]:
# SVM MODEL
import pickle
ngram_range = [(1, 1), (1, 2)]
C_list = [0.1, 1.0, 10]
loss_list = ['hinge', 'squared_hinge']
try_SVM(df1, ngram_range, loss_list, C_list)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten, Dropout
from keras.layers.pooling import MaxPooling1D
from keras.layers.convolutional import Conv1D
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
import gensim
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import keras



Using TensorFlow backend.


In [0]:
# model hyper parameters
EMBEDDING_DIM = 100
SEQUENCE_LENGTH_PERCENTILE = 90
n_layers = 2
hidden_units = 500
batch_size = [50, 100, 200]
pretrained_embedding = False
# if we have pre-trained embeddings, specify if they are static or non-static embeddings
TRAINABLE_EMBEDDINGS = True
patience = [2, 4, 6]
dropout_rate = 0.3
n_filters = 100
window_size = [8, 10]
dense_activation = "relu"
l2_penalty = [0.0003, 0.001, 0.01]
epochs = 10
VALIDATION_SPLIT = 0.1


In [0]:
def token_to_index(token, dictionary):
    """
    Given a token and a gensim dictionary, return the token index
    if in the dictionary, None otherwise.
    Reserve index 0 for padding.
    """
    if token not in dictionary.token2id:
        return None
    return dictionary.token2id[token] + 1

def texts_to_indices(text, dictionary):
    """
    Given a list of tokens (text) and a gensim dictionary, return a list
    of token ids.
    """
    result = list(map(lambda x: token_to_index(x, dictionary), text))
    return list(filter(None, result))


def train(train_texts, train_labels, dictionary, batch_size, window_size, l2_penalty, patience, model_file=None, EMBEDDINGS_MODEL_FILE=None):
    """
    Train a word-level CNN text classifier.
    :param train_texts: tokenized and normalized texts, a list of token lists, [['sentence', 'blah', 'blah'], ['sentence', '2'], .....]
    :param train_labels: the label for each train text
    :param dictionary: A gensim dictionary object for the training text tokens
    :param model_file: An optional output location for the ML model file
    :param EMBEDDINGS_MODEL_FILE: An optinal location for pre-trained word embeddings file location
    :return: the produced keras model, the validation accuracy, and the size of the training examples
    """
    assert len(train_texts)==len(train_labels)
    # compute the max sequence length
    # why do we need to do that?
    lengths=list(map(lambda x: len(x), train_texts))
    a = np.array(lengths)
    MAX_SEQUENCE_LENGTH = int(np.percentile(a, SEQUENCE_LENGTH_PERCENTILE))
    # convert all texts to dictionary indices
    train_texts_indices = list(map(lambda x: texts_to_indices(x, dictionary), train_texts))
    # pad or truncate the texts
    x_data = pad_sequences(train_texts_indices, maxlen=int(MAX_SEQUENCE_LENGTH))
    # convert the train labels to one-hot encoded vectors
    train_labels = keras.utils.to_categorical(train_labels)
    y_data = train_labels

    model = Sequential()

    # create embeddings matrix from word2vec pre-trained embeddings, if provided
    if pretrained_embedding:
        embeddings_index = gensim.models.KeyedVectors.load_word2vec_format(EMBEDDINGS_MODEL_FILE, binary=True)
        embedding_matrix = np.zeros((len(dictionary) + 1, EMBEDDING_DIM))
        for word, i in dictionary.token2id.items():
            embedding_vector = embeddings_index[word] if word in embeddings_index else None
            if embedding_vector is not None:
                # words not found in embedding index will be all-zeros.
                embedding_matrix[i] = embedding_vector
        model.add(Embedding(len(dictionary) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=TRAINABLE_EMBEDDINGS))
    else:
        model.add(Embedding(len(dictionary) + 1,
                            EMBEDDING_DIM,
                            input_length=MAX_SEQUENCE_LENGTH))
    # add drop out for the input layer, why do you think this might help?
    model.add(Dropout(dropout_rate))
    # add a 1 dimensional conv layer
    # a rectified linear activation unit, returns input if input > 0 else 0
    model.add(Conv1D(filters=n_filters,
                     kernel_size=window_size,
                     activation='relu'))
    # add a max pooling layer
    model.add(MaxPooling1D(MAX_SEQUENCE_LENGTH - window_size + 1))
    model.add(Flatten())

    # add 0 or more fully connected layers with drop out
    for _ in range(n_layers):
        model.add(Dropout(dropout_rate))
        model.add(Dense(hidden_units,
                        activation=dense_activation,
                        kernel_regularizer=l2(l2_penalty),
                        bias_regularizer=l2(l2_penalty),
                        kernel_initializer='glorot_uniform',
                        bias_initializer='zeros'))

    # add the last fully connected layer with softmax activation
    model.add(Dropout(dropout_rate))
    model.add(Dense(len(train_labels[0]),
                    activation='softmax',
                    kernel_regularizer=l2(l2_penalty),
                    bias_regularizer=l2(l2_penalty),
                    kernel_initializer='glorot_uniform',
                    bias_initializer='zeros'))

    # compile the model, provide an optimizer
    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])

    # print a summary
    print(model.summary())

    # train the model with early stopping
    early_stopping = EarlyStopping(patience=patience)
    Y = np.array(y_data)

    fit = model.fit(x_data,
                    Y,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=VALIDATION_SPLIT,
                    verbose=1,
                    callbacks=[early_stopping])

    print(fit.history.keys())
    val_accuracy = fit.history['acc'][-1]

    print(val_accuracy)
    # save the model

    if model_file:
        model.save(model_file)
    return model, val_accuracy, len(train_labels)


In [0]:
mydict = gensim.corpora.Dictionary(stemmed_tokens)


for w in window_size:
  for j in batch_size:
    for p in patience:
      for l in l2_penalty:
        model_file = "cnn_{0}_{1}_{2}_{3}.model".format(w,j, p, l)
        print("\n\n\n\n\n\n  The following is for window size of {0} and batch size of {1} and patience of {2} and l2 penalty of {3}".format(w,j, p, l))
        train(stemmed_tokens, rating_cat_list, mydict, batch_size=j, window_size=w, l2_penalty=l, patience=p, model_file=model_file)


In [0]:
mydict = gensim.corpora.Dictionary(stemmed_tokens)

mydict.save('cnn.dict')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
x=[2]

In [0]:
x[0]==2

True